In [24]:
pip install pyaudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyaudio)


In [25]:
!pip install SpeechRecognition googletrans==4.0.0-rc1

In [26]:
!pip install ffmpeg-python

In [48]:
# @title Convert input speech data to text data (any language to english)

import speech_recognition as sr
import wave
import numpy as np
import io
import ffmpeg
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from googletrans import Translator

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
    display(HTML(AUDIO_HTML))
    data = eval_js("data")
    if not data:
        return None, None, "No audio data received", None

    binary = b64decode(data.split(',')[1])
    if not binary:
        return None, None, "Audio data could not be decoded", None

    # Convert base64 to WAV
    process = (ffmpeg
      .input('pipe:0')
      .output('pipe:1', format='wav')
      .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)
    if not output:
        return None, None, "Audio data could not be converted to WAV", None

    # Read WAV file
    try:
        with wave.open(io.BytesIO(output), 'rb') as wf:
            sr_rate = wf.getframerate()
            audio = np.frombuffer(wf.readframes(wf.getnframes()), dtype=np.int16)
    except Exception as e:
        return None, None, f"Error reading WAV file: {e}", None

    # Convert audio to text
    recognizer = sr.Recognizer()
    audio_data = sr.AudioData(output, sr_rate, 2)
    translator = Translator()
    try:
        text = recognizer.recognize_google(audio_data, show_all=True)
        if not isinstance(text, str):  # If Google returns a dict (for multiple hypotheses)
            text = text.get('alternative', [{}])[0].get('transcript', '')
    except sr.UnknownValueError:
        text = "Google Speech Recognition could not understand the audio"
    except sr.RequestError as e:
        text = f"Could not request results from Google Speech Recognition service; {e}"

    # Detect language and translate text to English if not already
    try:
        detected_lang = translator.detect(text).lang
        if detected_lang != 'en':
            text = translator.translate(text, src=detected_lang, dest='en').text
    except Exception as e:
        detected_lang = "unknown"
        text = f"Translation failed; {e}"

    return audio, sr_rate, text, detected_lang

In [49]:
audio, sr_rate, text, detected_lang = get_audio()
print("Recognized text:", text)
print("Detected language:", detected_lang)

Recognized text: Google Morning Song Baby suspicious of beauty tell me another call tomorrow
Detected language: en


In [50]:
input_sentence = text
input_sentence

'Google Morning Song Baby suspicious of beauty tell me another call tomorrow'

In [51]:
# @title Test data
test_data = [
    'Hello, this is Officer Jane Miller from the Federal Investigation Department. We are calling because we have detected some suspicious activity related to your Social Security number. It seems that someone has been using your identity to commit fraud/criminal activity. To avoid any legal consequences and to protect your assets, we need you to verify your personal information immediately. Press 1 to speak with an agent who can help you resolve this matter. Press 2 to enable video call. If you do not take action today, your bank accounts could be frozen, and you may face criminal charges. This is a very serious issue, so please act quickly.',  # Phishing
    'Dear customer, your order has been shipped and will arrive soon.',  # Not phishing
    'Your bank account has been compromised. Please verify your details now to avoid closure.Tell your account number.',  # Phishing
    'Join our webinar on cybersecurity next week!',  # Not phishing
    'Hurry you don\'t have time', # Phishing
    'I am good. I have a pet dog', # Not phishing
    'Police are coming after you to arrest you', # Phishing
    'What is your name', # Not phishing
]
test_labels = [1, 0, 1, 0, 1, 0, 1, 0]  # Actual labels for the test data

In [52]:
# @title Initialise and Train the model

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Convert text data to numerical data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(test_data)

# Stratified split to maintain the proportion of classes
X_train, X_test, y_train, y_test = train_test_split(X, test_labels, test_size=0.2, random_state=0, stratify=test_labels)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

Accuracy: 0.5
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
# @title Model Prediction

# Vectorize the new text
text_vectorized = vectorizer.transform([input_sentence])

# Predict using the trained model
prediction = model.predict(text_vectorized)

# Output the result
if prediction[0] == 1:
    message = "This message is likely a phishing attempt."
else:
    message = "This message is not phishing."

print(message)

This message is likely a phishing attempt.


In [54]:
pip install gtts

In [55]:
pip install playsound

In [ ]:
import os
print(os.getcwd())  # Print current working directory

/content


In [58]:
from gtts import gTTS
import pygame
import os
from googletrans import Translator

# Initialize the Translator
translator = Translator()

# Define the target language for translation (Kannada in this example)
#detected_lang = 'kn'

# Translating from source language (auto-detected) to the target language
translated_message = translator.translate(message, dest=detected_lang).text

# Create a gTTS object with the specified language
speech = gTTS(text=translated_message, lang=detected_lang)

# Save the converted audio to a file
speech.save('Voice_message.mp3')

In [ ]:
pip install openai

In [ ]:
pip install --upgrade httpx

In [ ]:
# @title Open-AI generated responce
import openai

# Configure OpenAI API
openai.api_key = 'your-api-key'

# Output the result and generate automated message if phishing
if message == "This message is likely a phishing attempt.":
  # Generate a warning message using GPT-4
  response = openai.Completion.create(
  engine="gpt-4o-mini",
  prompt="Generate a reply message to be given by the user to avoid phishing attempt. The message should be clear, concise, and authoritative.",
  max_tokens=100
  )
  warning_message = response.choices[0].text.strip()
  print("Warning Message Generated by GPT-4:")
  print(warning_message)

# Initialize the Translator
translator = Translator()

# Translating from source language (auto-detected) to the target language
translated_warning_message = translator.translate(warning_message, dest=detected_lang).text

# Create a gTTS object with the specified language
speech = gTTS(text=translated_warning_message, lang=detected_lang)

# Save the converted audio to a file
speech.save('Warning_voice_message.mp3')